# Run TAP (Dial's algorithm B) and recover sample paths

In [1]:
import os
os.environ['R_HOME'] = r"C:\Program Files\R\R-4.3.3"

import time
import gc
from collections import defaultdict

from pathlib import Path

import pandas as pd
import numpy as np

from rpy2.robjects.packages import importr
from rpy2.robjects import pandas2ri
from rpy2 import robjects
from rpy2.robjects.conversion import localconverter

# Convert pandas.DataFrames to R dataframes automatically.
pandas2ri.activate()

cpp_routing = importr("cppRouting")
r_cpp_parallel = importr("RcppParallel")

## Read in sample data

Some implementations of the TAP like the node numbers to start at 1, not 0.  For example, the 'Greedy' algorithm by Xie and Ni.  The Larmet implementation, [cppRouting](https://github.com/vlarmet/cppRouting), does not care.

However, cppRouting does not like loops and parallel arcs, so we eliminate them

In [2]:
data_directory = Path(r'C:\Users\Marc.Meketon\OneDrive - MMC\Documents\OliverWyman\DOE_IntermodalRouting')
od_df = pd.read_parquet(data_directory / 'tap_highway_tons.parquet')

#  'from' is a keyword in Python and makes referring to fields with that name harder.
od_df.rename(columns={'from': 'From', 'to': 'To', 'tons': 'Tons'}, inplace=True)

# some software would like node numbers to begin with '1'; not the cppRouting code, but others
od_df.From += 1
od_df.To += 1
tap_network_df = pd.read_parquet(data_directory / 'tap_network_dataframe.parquet')

# 'head' and 'tail' are pandas functions, so tap_network_df.head does not work to refer to the column
#   but tap_network_df.Head can work
tap_network_df.rename(columns={'tail': 'Tail', 'head': 'Head'}, inplace=True)
tap_network_df.Tail += 1
tap_network_df.Head += 1

print(f'od_df.shape: {od_df.shape}, tap_network_df.shape: {tap_network_df.shape}')

od_df.shape: (965, 3), tap_network_df.shape: (638527, 8)


In [3]:
tap_network_df = tap_network_df[~tap_network_df.duplicated(subset=['Tail','Head'])]
print(f'tap_network_df.shape after removing parallel links: {tap_network_df.shape}')

tap_network_df = tap_network_df[tap_network_df.Tail != tap_network_df.Head]
print(f'tap_network_df.shape after loop links: {tap_network_df.shape}')

tap_network_df['IDX'] = list(range(1, len(tap_network_df) + 1))
tap_network_df.set_index(keys='IDX', inplace=True, drop=False)

tap_network_df.shape after removing parallel links: (637451, 8)
tap_network_df.shape after loop links: (637424, 8)


In [4]:
# reduce TAP capacity by 25% so that multiple paths would be possible
tap_network_df.capacity *= 0.75

In [5]:
tap_network_df

,Tail,Head,length,speed,fft,beta,alpha,capacity,IDX
IDX,,,,,,,,,
1,1,2,0.054083,43.0,0.001258,4,0.15,60000.0,1
2,1,4,0.026030,25.0,0.001041,4,0.15,60000.0,2
3,1,9,0.092914,34.0,0.002733,4,0.15,60000.0,3
4,2,3,0.114854,43.0,0.002671,4,0.15,60000.0,4
5,2,4,0.032432,43.0,0.000754,4,0.15,60000.0,5
...,...,...,...,...,...,...,...,...,...
637420,332475,332473,0.162602,25.0,0.006504,4,0.15,60000.0,637420
637421,332475,332474,0.211961,29.5,0.007185,4,0.15,60000.0,637421
637422,332476,332357,1.073753,25.0,0.042950,4,0.15,60000.0,637422


## Run the TAP algorithm and obtain link flows

In [6]:
def calc_tap_link_volumes(network_df: pd.DataFrame, ods_df: pd.DataFrame) -> tuple[float, float, int, pd.DataFrame]:
    # NOTE:  do not allow parallel edges.  This is a limitation of the cppRouting package
    sgr = cpp_routing.makegraph(network_df[["Tail", "Head", "fft"]],
                            directed=True,
                            capacity = tap_network_df['capacity'],
                            alpha=tap_network_df['alpha'],
                            beta=tap_network_df['beta'])
    
    # save link from/to to link idx dictionary,
    link_from_to_2_idx = {(o, d): idx for o, d, idx in network_df[["Tail", "Head", "IDX"]].to_numpy()}

    max_gap = 1e-6   # default value was 0.001
    start_time = time.perf_counter()

    # we use the **{} way of passing keywords in because the arguement 'from' is reserved word
    #  and in this case Python is not happy with using 'from=ods_df.From'
    traffic = cpp_routing.assign_traffic(**{'Graph': sgr, 
                                            'from': ods_df.From,
                                            'to': ods_df.To,
                                            'demand': ods_df.Tons,
                                            'max_gap': max_gap,
                                            'algorithm': 'dial',
                                            'verbose': True})
    elapsed_time = time.perf_counter() - start_time

    gap = traffic.rx2('gap')[0]
    num_iterations = traffic.rx2('iteration')[0]
    with localconverter(robjects.default_converter + pandas2ri.converter):
        link_volumes_df = robjects.conversion.rpy2py(traffic.rx2('data'))
    link_volumes_df = link_volumes_df.astype({'from': 'int64', 'to': 'int64'})
    link_volumes_df['IDX'] = [link_from_to_2_idx[o, d] for o, d in link_volumes_df[["from", "to"]].to_numpy()]
    link_volumes_df.set_index(keys='IDX', inplace=True, drop=False)

    return elapsed_time, gap, num_iterations, link_volumes_df

In [7]:
elapsed_time, gap, num_iters, link_vol_df = calc_tap_link_volumes(tap_network_df, od_df)
print(f'Elapsed time (seconds): {round(elapsed_time,2)}')
print(f'gap: {gap}')
print(f'number of iterations: {num_iters}')
link_vol_df[link_vol_df.flow > 0.0].head()

Bushes initialization...
Iterating...
iteration 1 : 2.44567e-07
Elapsed time (seconds): 64.05
gap: 2.4456741876122123e-07
number of iterations: 1


,from,to,ftt,cost,flow,capacity,alpha,beta,IDX
IDX,,,,,,,,,
11612,5759,120,0.028355,0.028363,12244.610871,60000.0,0.15,4.0,11612
261,120,121,0.007449,0.007450,12244.610871,60000.0,0.15,4.0,261
262,121,122,0.086321,0.086343,12244.610871,60000.0,0.15,4.0,262
11581,5741,128,0.009471,0.009474,12244.610871,60000.0,0.15,4.0,11581
3492,1717,1716,0.018646,0.019493,44503.168033,60000.0,0.15,4.0,3492


In [8]:
# place the flows into an array in the order of the links (using IDX as the ordering)
tap_flow = np.zeros(len(link_vol_df) + 1)
tap_flow[link_vol_df.IDX.to_numpy()] = link_vol_df.flow
tap_flow[tap_flow > 0.0]

array([12244.61087137, 12244.61087137, 12244.61087137, ...,
        1076.99535348,  1076.99535348,  1076.99535348])

In [9]:
# place the costs into an array in the order of the links.  This is used for speeding up the path costs calculations

costs = np.zeros(len(link_vol_df) + 1)
costs[link_vol_df.IDX] = link_vol_df.cost
costs[[11612, 261, 262, 11581, 3492]]

array([0.02836252, 0.00745046, 0.08634325, 0.00947351, 0.01949257])

## Create a set of paths, possible some alternate paths as well

This section has two parts:
  1.  Find one path for each OD using the calculated costs from TAP assignment
  2.  For any path going through links whose total flow is greater than the TAP link flow, find an alternate path.

In [10]:
# ODPath contains the links for a particular path
class ODPath:
    def __init__(self, orig: int, dest: int, path_cost: float, flow: float, link_path: np.array) -> None:
        self.orig = orig
        self.dest = dest
        self.path_cost = path_cost
        self.flow = flow
        self.link_path = link_path
    
    def __repr__(self):
        return f'{self.orig}->{self.dest} ({np.round(self.path_cost,2)},{np.round(self.flow,2)}){self.link_path}'
    
    @property
    def od(self) -> str:
        return f'{self.orig}->{self.dest}'
    
# ODPaths contains a list of paths for the same OD
class ODPaths:
    def __init__(self, total_demand, initial_od_path) -> None:
        self.total_demand = total_demand
        self.list_of_paths: list[ODPath] = [initial_od_path]

In [11]:
# Make a network using the link costs derived from the TAP problem (ftt * (1 + alpha (flow/capacity)^beta))
# Then get the shortest path for all the OD's and store them

def get_sample_paths(ods_df: pd.DataFrame, link_volumes_df: pd.DataFrame) -> dict[tuple[int, int], ODPaths]:
    ret_paths: dict[tuple[int, int], ODPaths] = dict()
    # same links as original network, but use the cost that based on the user equilibrium flows
    sgr = cpp_routing.makegraph(link_volumes_df[['from', 'to', 'cost']])
    od_paths = cpp_routing.get_path_pair(**{'Graph': sgr,
                                        'from': ods_df.From,
                                        'to': ods_df.To,
                                        'algorithm': 'bi'})
    link_from_to_2_idx: dict[tuple[int, int], int] = {(o, d): idx for o, d, idx in link_volumes_df[["from", "to", "IDX"]].to_numpy()}
    od_2_flow: dict[tuple[int, int], float] = {(int(o), int(d)): flow for o, d, flow in ods_df[["From", "To", "Tons"]].to_numpy()}
    costs = np.zeros(len(link_volumes_df) + 1)
    costs[link_volumes_df.IDX] = link_volumes_df.cost
    for idx, o_d in enumerate(od_paths.names):
        orig, dest = (int(n) for n in o_d.split('_'))
        node_path = [int(n) for n in od_paths[idx]]
        link_path = np.array([link_from_to_2_idx[si, sip1] for si, sip1 in zip(node_path[:-1], node_path[1:])])
        path_cost = np.sum(costs[link_path])
        total_demand = od_2_flow[orig, dest]
        ret_paths[orig, dest] = ODPaths(total_demand, ODPath(orig, dest, path_cost, total_demand, link_path))
    return ret_paths

In [12]:
# run it.  This gets the first set of paths, one per OD

od_paths = get_sample_paths(od_df, link_vol_df)

### Compute secondary paths

Calculate the link flows if each of the paths had the full demand

Take away links that have too much flow, and then recalculate the shortest paths on the reduced network

In [13]:
# Compute link flows based on the paths found above

def calc_link_flows(paths: dict[tuple[int, int], ODPaths]):
    link_flows = np.zeros(len(tap_network_df) + 1)
    for od_paths in paths.values():
        for path in od_paths.list_of_paths:
            link_flows[path.link_path] += path.flow
    return link_flows

In [14]:
flow_by_link = calc_link_flows(od_paths)
flow_by_link[flow_by_link > 0.0]

array([12244.61087137, 12244.61087137, 12244.61087137, ...,
        1076.99535348,  1076.99535348,  1076.99535348])

In [15]:
# identify all links whose flow is greater than the TAP flow

idx_where_flow_violates_tap = np.asarray(flow_by_link > tap_flow + 0.1).nonzero()[0]
link_ratios = np.ones(len(tap_network_df) + 1)
link_ratios[idx_where_flow_violates_tap] = tap_flow[idx_where_flow_violates_tap] / flow_by_link[idx_where_flow_violates_tap]
idx_where_flow_violates_tap_df = pd.DataFrame({'IDX': idx_where_flow_violates_tap, 
                                               'SP_FLOW': flow_by_link[idx_where_flow_violates_tap],
                                               'TAP_FLOW': tap_flow[idx_where_flow_violates_tap],
                                               'RATIO': tap_flow[idx_where_flow_violates_tap] / flow_by_link[idx_where_flow_violates_tap]})
idx_where_flow_violates_tap_df

,IDX,SP_FLOW,TAP_FLOW,RATIO
0,67060,36315.893614,8135.178892,0.224012
1,84022,36315.893614,8135.178892,0.224012
2,84023,33605.730803,8512.435947,0.253303
3,84024,33605.730803,8512.435947,0.253303
4,84025,36315.893614,8135.178892,0.224012
...,...,...,...,...
128,399287,12363.346334,3065.947702,0.247987
129,399303,12363.346334,3065.947702,0.247987
130,560100,8705.957203,2812.846699,0.323094
131,560114,8705.957203,2812.846699,0.323094


In [16]:
# Essentially 'transpose' the od_paths structure so that for each link we know which paths go through it

paths_through_links = np.empty(len(tap_network_df) + 1, dtype=object)
for idx in range(len(paths_through_links)):
    paths_through_links[idx] = []
for paths in od_paths.values():
    for path in paths.list_of_paths:
        for link_idx in path.link_path:
            paths_through_links[link_idx].append(path)

In [17]:
# identify all paths that use those links whose flow is greater than the TAP Flow
paths_that_overflow = {path for paths in paths_through_links[idx_where_flow_violates_tap] for path in paths}
[path.od for path in paths_that_overflow]

['126337->125900',
 '12485->34492',
 '34492->1716',
 '126337->131570',
 '179821->208964',
 '126337->126812',
 '179753->208964',
 '34492->12485',
 '126337->126786',
 '206759->208964',
 '291895->289927',
 '1716->34492',
 '291895->288630',
 '125900->126786',
 '181365->208964']

In [18]:
# Let's flow this on a network where the links tap-flow is greater than the shortest path flows
#   which means removing links where the path flows are too large

reduced_tap_network_df = tap_network_df[~tap_network_df.IDX.isin(idx_where_flow_violates_tap)].copy()
reduced_tap_network_df = reduced_tap_network_df.merge(link_vol_df[['from', 'to', 'cost']], left_on=['Tail', 'Head'], right_on=['from', 'to'])
reduced_tap_network_df

,Tail,Head,length,speed,fft,beta,alpha,capacity,IDX,from,to,cost
0,1,2,0.054083,43.0,0.001258,4,0.15,60000.0,1,1,2,0.001258
1,1,4,0.026030,25.0,0.001041,4,0.15,60000.0,2,1,4,0.001041
2,1,9,0.092914,34.0,0.002733,4,0.15,60000.0,3,1,9,0.002733
3,2,3,0.114854,43.0,0.002671,4,0.15,60000.0,4,2,3,0.002671
4,2,4,0.032432,43.0,0.000754,4,0.15,60000.0,5,2,4,0.000754
...,...,...,...,...,...,...,...,...,...,...,...,...
637286,332475,332473,0.162602,25.0,0.006504,4,0.15,60000.0,637420,332475,332473,0.006504
637287,332475,332474,0.211961,29.5,0.007185,4,0.15,60000.0,637421,332475,332474,0.007185
637288,332476,332357,1.073753,25.0,0.042950,4,0.15,60000.0,637422,332476,332357,0.042950
637289,332476,332471,0.101603,29.5,0.003444,4,0.15,60000.0,637423,332476,332471,0.003444


## Run shortest path on reduced network to find potential secondary paths

Any secondary path that is found must have the same path-cost as the first set of paths.

It could be that a secondary path has a higher cost - that could occur because we are too aggressive in taking out links, which is due to having not figuring out precisely how to reduce the flows in paths that use those links that have high flow.

We only keep those secondary paths whose cost is the same as the primary path.  Right now, if the secondary path should have costs within 1% of the primary path.

In [19]:
# same links as original network, but use the cost that based on the user equilibrium flows

sgr = cpp_routing.makegraph(reduced_tap_network_df[['Tail', 'Head', 'cost']])
reduced_od_paths = cpp_routing.get_path_pair(**{'Graph': sgr,
                                    'from': [path.orig for path in paths_that_overflow],
                                    'to': [path.dest for path in paths_that_overflow],
                                    'algorithm': 'bi'})
link_from_to_2_idx: dict[tuple[int, int], int] = {(o, d): idx for o, d, idx in reduced_tap_network_df[["from", "to", "IDX"]].to_numpy()}

costs = np.zeros(len(tap_network_df) + 1)
costs[reduced_tap_network_df.IDX] = reduced_tap_network_df.cost
for idx, o_d in enumerate(reduced_od_paths.names):
    orig, dest = (int(n) for n in o_d.split('_'))
    node_path = [int(n) for n in reduced_od_paths[idx]]
    link_path = np.array([link_from_to_2_idx[si, sip1] for si, sip1 in zip(node_path[:-1], node_path[1:])])
    path_cost = np.sum(costs[link_path])
    # print(ODPath(orig, dest, path_cost, 0, link_path))
    original_path_cost = od_paths[orig, dest].list_of_paths[0].path_cost
    relative_difference = (path_cost - original_path_cost)/original_path_cost
    print(f'Original path cost: {original_path_cost}, relative difference: {(path_cost - original_path_cost)/original_path_cost}')
    if relative_difference <= 0.01:
        od_paths[orig, dest].list_of_paths.append(ODPath(orig, dest, path_cost, 0.0, link_path))


Original path cost: 2.5927138626100374, relative difference: 0.02806015014139676
Original path cost: 2.7341004093316186, relative difference: 1.2594121695206018e-05
Original path cost: 1.0160212155707553, relative difference: 1.901326501103208e-14
Original path cost: 2.9581606167038466, relative difference: 0.03795074251421588
Original path cost: 1.1634611713048546, relative difference: 6.330289245225671e-07
Original path cost: 2.4111895761462225, relative difference: 0.030403643876065554
Original path cost: 1.1694226338710678, relative difference: 6.29801880761378e-07
Original path cost: 2.758579656947695, relative difference: 6.922343523942969e-15
Original path cost: 1.9465504424043063, relative difference: 3.853858775008427e-08
Original path cost: 0.9291444047771013, relative difference: 7.926696543941499e-07
Original path cost: 0.6990558630256193, relative difference: 4.42137747462049e-10
Original path cost: 0.9977717431529756, relative difference: 3.4510491521060974e-05
Original p